In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Conv2D, MaxPooling2D, Flatten, Dense
%matplotlib inline 

In [3]:
input_shape = (250, 250, 3)
batch = 8
lr = 0.002

In [4]:
img_gen = ImageDataGenerator(rotation_range=5,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            rescale=1/255,
                            fill_mode='nearest')

In [5]:
train_gen = img_gen.flow_from_directory('dataset/train',
                                   target_size=input_shape[:2],
                                   batch_size=batch,
                                   class_mode='categorical',
                                   shuffle=True,
                                   seed=42)
test_gen = img_gen.flow_from_directory('dataset/test',
                                        target_size=input_shape[:2],
                                        batch_size=batch,
                                        class_mode='categorical',
                                        shuffle=True,
                                        seed=42)

Found 9867 images belonging to 3 classes.
Found 5247 images belonging to 3 classes.


In [6]:
train_gen.class_indices

{'COVID-19': 0, 'normal': 1, 'pneumonia': 2}

In [7]:
test_gen.class_indices

{'COVID-19': 0, 'normal': 1, 'pneumonia': 2}

In [8]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(2,2), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(2,2), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=32,kernel_size=(2,2), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('selu'))

model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae', 'acc'])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 249, 249, 32)      416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 124, 124, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 123, 123, 64)      8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 61, 61, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 32)        8224      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28800)            

In [10]:
results = model.fit_generator(train_gen, epochs = 100, steps_per_epoch=250, 
                             validation_data=test_gen, validation_steps=20)


Epoch 1/100
250/250 [==============================] - 118s 473ms/step - loss: 0.1356 - mae: 0.2364 - acc: 0.7323 - val_loss: 0.0310 - val_mae: 0.1797 - val_acc: 0.8188
Epoch 2/100
250/250 [==============================] - 113s 450ms/step - loss: 0.1014 - mae: 0.1938 - acc: 0.7895 - val_loss: 0.1721 - val_mae: 0.1780 - val_acc: 0.7625
Epoch 3/100
250/250 [==============================] - 110s 442ms/step - loss: 0.1027 - mae: 0.1933 - acc: 0.7975 - val_loss: 0.1194 - val_mae: 0.1872 - val_acc: 0.7250
Epoch 4/100
  8/250 [..............................] - ETA: 2:04 - loss: 0.1215 - mae: 0.1783 - acc: 0.8125

KeyboardInterrupt: 